In [1]:
from qutip import *
import matplotlib.pyplot as plt
import numpy as np
from helper_code_qutip import *
from scipy import integrate
import scipy.io
import time

In [ ]:
N = 3
d = 2**N
dims = [2]*N

w0list = np.linspace(1,1,N)
glist = np.linspace(0.0016,0.0016,N-1)

delta = 1

H_S = create_hamiltonian(w0list,glist,delta,N)
eigenergies,eigstates=H_S.eigenstates()
number = len(eigenergies)

mu2 = 0
gamma1 = 1
tb = 0.01

In [21]:
create_sm_list = [create_sm(N,i + 1) for i in range(N)]
beta = 1
rho_th = (-beta*H_S).expm()/((-beta*H_S).expm()).tr() #left thermal density matrix
limit_value = 700
b = 500

integral11=np.empty((number,number),dtype=np.cdouble) #stores J * N integral for left bath
integral12=np.empty((number,number),dtype=np.cdouble) # stores J integral (just to check) for the left bath
integral21=np.empty((number,number),dtype=np.cdouble) #stores J*N integral for right bath
integral22=np.empty((number,number),dtype=np.cdouble)

        #print("Integral calculations at beta2 = {} and e = {} are : ".format(beta2,e))

for i in range(number):
    for k in range(number):
                freq=eigenergies[k]-eigenergies[i]
                #print(f"Absolute frequency  for i = {i}, k = {k} is ",np.absolute(freq))
                #print(i,k,freq)
                if( np.absolute(freq) >= 1/10**10):
                    integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu2,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                    integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #left bath done
                    """integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                    integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #right bath"""
        
                if (np.absolute(freq)<=1/10**10):  #The problem is arising here....
                    integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu2,gamma1),limit=limit_value)[0]
                    integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma1),limit=limit_value)[0]
                    """integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value)[0]
                    integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma2),limit=limit_value)[0]"""
                
            
            #expected=1.0j*(eigenergies[k]-eigenergies[i])/(2*tb*tb)
        #        print(i,k,integral2[i,k],expected)
    
    
        # PAY ATTENTION TO THE WAY THESE COEFFICIENTS ARE BEING COMPUTED
    
constant12=np.empty((number,number),dtype=np.cdouble)
constant11=np.empty((number,number),dtype=np.cdouble)
"""constant21=np.empty((number,number),dtype=np.cdouble)
constant22=np.empty((number,number),dtype=np.cdouble)"""
        
        
        
for i in range(number):
        for k in range(number):
                constant12[i,k]=integral12[i,k]+integral11[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma1)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu2,gamma1))    #full coefficient created this is nbar+1
                constant11[i,k]=integral11[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta,mu2,gamma1)                                       # the full coefficient is created
                
                """constant22[i,k]=integral22[i,k]+integral21[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma2)+func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2))    #full coefficient created this is nbar+1
                constant21[i,k]=integral21[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2)   # the full coefficient is created"""

C:\Users\DELL\AppData\Local\Temp\ipykernel_30096\3859668315.py:26: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu2,gamma1),limit=limit_value)[0]


In [ ]:
epsilon = 0.01

## Now we will write out the matrix elements

A = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        sum = 0
        vi = eigstates[i]
        vk = eigstates[k]
        proj_i = vi*vi.dag()
        proj_k = vk*vk.dag()
        for y in range(number):
            for l in range(N-1):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list[l]*proj_y,create_sm_list[l].dag())*constant11[k,y]
                sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list[l].dag(),proj_y*create_sm_list[l]*proj_k)*constant12[y,k]
                sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

        A[i,k] = sum

In [23]:
b = np.zeros((number),dtype=complex)
q,r = np.linalg.qr(A.T)

#print(r)
diag_r = [r[i][i] for i in range(number)]
print(diag_r)

[(-0.0001944942609010382+0j), (-0.0001984938002483354+0j), (-0.00018450330875771545+0j), (-0.00017543290814876257+0j), (-0.00018326378803179048+0j), (-0.00019493573711438407+0j), (-0.000208399076421805+0j), (-0.00020036123017742646+0j), (-0.00021046887584018768+0j), (-0.00020862619731920376+0j), (-0.00020049017303204084+0j), (-0.0001859827251328909+0j), (-0.00021919326635959393+0j), (-0.0002260823591652279+0j), (-0.00020461821961359006+0j), (-2.503775923358397e-21+0j)]


In [24]:
rho_th_new = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        vi = eigstates[i]
        vk = eigstates[k]

        rho_th_new[i,k] = vi.dag()*rho_th*vk

#print(rho_th_new)

rho_th_diag = [rho_th_new[i,i] for i in range(number)]

In [25]:
A_new = A[:-1]
A_new = np.vstack([A_new,np.ones((1,number))])
b[-1] = 1  ## Last element of b is 1

x = np.linalg.solve(A_new,b)
print(np.dot(A_new,x))
print(np.dot(A[-1],x))
x_real = [np.real(x[i]) for i in range(number)]

rho = np.diag(x_real)

#set U matrix whose columns are the eigenvectors of the Hamiltonian

U = np.zeros((number,number),dtype=complex)
for i in range(number):
    U[:,i] = eigstates[i].full().flatten()

print([x.real for x in  rho_th_diag])
print(x_real)

[ 3.38813179e-21-2.82118644e-37j  0.00000000e+00+2.35098870e-38j
 -2.54109884e-21+0.00000000e+00j  0.00000000e+00+2.82118644e-37j
  2.54109884e-21-9.40395481e-38j -2.54109884e-21+0.00000000e+00j
  0.00000000e+00+9.40395481e-38j  8.47032947e-22+0.00000000e+00j
 -8.47032947e-22-1.17549435e-38j  1.69406589e-21+1.17549435e-38j
 -2.11758237e-21-2.82118644e-37j  2.11758237e-22+3.52648305e-38j
  8.47032947e-22+0.00000000e+00j -1.69406589e-21+9.40395481e-38j
  1.69406589e-21-4.70197740e-38j  1.00000000e+00-3.08148791e-33j]
(-6.670384459627615e-21-8.683227962369852e-22j)
[0.2867104807443116, 0.1054748914342129, 0.10527736219305431, 0.10480200765390596, 0.10432879946353746, 0.0388020441184368, 0.03872937717158434, 0.038644926216399096, 0.03855450400936415, 0.038380420444733665, 0.03821890796739024, 0.014274474306600175, 0.014247741630800463, 0.014183409389607018, 0.014119367625133614, 0.005251285630928183]
[0.2867104807443116, 0.1054748914342129, 0.10527736219305427, 0.10480200765390597, 0.10432

In [4]:

def L2_red(rho,eigstates):
    sum = 0
    rho = Qobj(rho)
    rho.dims = [dims,dims]
    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()

            for l in range(N):
                op = commutator(rho*proj_i*create_sm_list[l]*proj_k,create_sm_list[l].dag())*constant11[i,k] + commutator(create_sm_list[l].dag(),proj_i*create_sm_list[l]*proj_k*rho)*constant12[i,k]
                sum += op
                sum += op.dag()

            """for l in range(NL2):
                op = commutator(rho*proj_i*create_sm_list_right[l]*proj_k,create_sm_list_right[l].dag())*constant21[i,k] + commutator(create_sm_list_right[l].dag(),proj_i*create_sm_list_right[l]*proj_k*rho)*constant22[i,k]
                sum += op
                sum += op.dag()"""
    data = sum.full()
    sum = np.array(data,dtype=complex)
    return sum

In [25]:
N = 5
c = 3
d = 2**N
dims = [2]*N
betalist = [1,1.2,1.5,1.7,2.0,2.2,2.5,2.7,3.0,3.5]
gvals = np.logspace(-3,0,10)

for index in range(len(gvals)):
    print(f"Starting with g = {gvals[index]}")
    g = gvals[index]
    w0list = np.linspace(1,1,N)
    glist = np.linspace(g,g,N-1)

    delta = 1

    H_S = create_hamiltonian(w0list,glist,delta,N)
    eigenergies,eigstates=H_S.eigenstates()
    number = len(eigenergies)

    mu2 = 0
    gamma1 = 1
    tb = 0.01

    create_sm_list = [create_sm(N,i + 1) for i in range(N)]
    beta = 1
    rho_th = (-beta*H_S).expm()/((-beta*H_S).expm()).tr() #left thermal density matrix
    limit_value = 700
    f = 500
    b = np.zeros((number),dtype=complex)

    integral11=np.empty((number,number),dtype=np.cdouble) #stores J * N integral for left bath
    integral12=np.empty((number,number),dtype=np.cdouble) # stores J integral (just to check) for the left bath
    integral21=np.empty((number,number),dtype=np.cdouble) #stores J*N integral for right bath
    integral22=np.empty((number,number),dtype=np.cdouble)

            #print("Integral calculations at beta2 = {} and e = {} are : ".format(beta2,e))

    for i in range(number):
        for k in range(number):
                    freq=eigenergies[k]-eigenergies[i]
                    #print(f"Absolute frequency  for i = {i}, k = {k} is ",np.absolute(freq))
                    #print(i,k,freq)
                    if( np.absolute(freq) >= 1/10**10):
                        integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,f,args=(tb,beta,mu2,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                        integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,f,args=(tb,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #left bath done
                        """integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                        integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #right bath"""
            
                    if (np.absolute(freq)<=1/10**10):  #The problem is arising here....
                        integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,f,args=(tb,beta,mu2,gamma1),limit=limit_value)[0]
                        integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,f,args=(tb,gamma1),limit=limit_value)[0]
                        """integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value)[0]
                        integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma2),limit=limit_value)[0]"""
                    
                
                #expected=1.0j*(eigenergies[k]-eigenergies[i])/(2*tb*tb)
            #        print(i,k,integral2[i,k],expected)
        
        
            # PAY ATTENTION TO THE WAY THESE COEFFICIENTS ARE BEING COMPUTED
        
    constant12=np.empty((number,number),dtype=np.cdouble)
    constant11=np.empty((number,number),dtype=np.cdouble)
    """constant21=np.empty((number,number),dtype=np.cdouble)
    constant22=np.empty((number,number),dtype=np.cdouble)"""
            
            
            
    for i in range(number):
            for k in range(number):
                    constant12[i,k]=integral12[i,k]+integral11[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma1)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu2,gamma1))    #full coefficient created this is nbar+1
                    constant11[i,k]=integral11[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta,mu2,gamma1)                                       # the full coefficient is created
                    
                    """constant22[i,k]=integral22[i,k]+integral21[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma2)+func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2))    #full coefficient created this is nbar+1
                    constant21[i,k]=integral21[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2)   # the full coefficient is created"""

    epsilon = 0.01

    ## Now we will write out the matrix elements

    A = np.zeros((number,number),dtype=complex)

    for i in range(number):
        for k in range(number):
            sum = 0
            vi = eigstates[i]
            vk = eigstates[k]
            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()
            for y in range(number):
                for l in range(N-c):
                    proj_y = eigstates[y]*eigstates[y].dag()
                    op1 = commutator(proj_k*create_sm_list[l]*proj_y,create_sm_list[l].dag())*constant11[k,y]
                    sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                    op2 = commutator(create_sm_list[l].dag(),proj_y*create_sm_list[l]*proj_k)*constant12[y,k]
                    sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

            A[i,k] = sum

    rho_th_new = np.zeros((number,number),dtype=complex)

    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            rho_th_new[i,k] = vi.dag()*rho_th*vk

    #print(rho_th_new)

    rho_th_diag = [rho_th_new[i,i] for i in range(number)]

    A_new = A[:-1]
    A_new = np.vstack([A_new,np.ones((1,number))])
    b[-1] = 1  ## Last element of b is 1

    x = np.linalg.solve(A_new,b)
    #print(np.dot(A_new,x))
    #print(np.dot(A[-1],x))
    x_real = [np.real(x[i]) for i in range(number)]

    rho = np.diag(x_real)

    #set U matrix whose columns are the eigenvectors of the Hamiltonian

    U = np.zeros((number,number),dtype=complex)
    for i in range(number):
        U[:,i] = eigstates[i].full().flatten()


    rho_comp2 = np.dot(U,np.dot(rho,U.T.conjugate()))
    L2_redfield = L2_red(rho_comp2,eigstates)
    rho_th_qutip = Qobj(rho_th)
    rho_th_qutip.dims = [dims,dims]

    rho_comp_qutip = Qobj(rho_comp2)
    rho_comp_qutip.dims = [dims,dims]
    print(tracedist(rho_th_qutip,rho_comp_qutip))

    rho_th_check = np.zeros((number,number),dtype=complex)
    for i in range(number):
        for k in range(number):
            rho_th_check[i,k] = rho_th[i,k]

    #print(rho_th_check)
    #print(rho_th_new)
    #print(L2_redfield)
    #print(rho_th_check)

    data_dict = {"dm_ness":rho_comp2, "dm_th":rho_th_new,"L2_red":L2_redfield, "dm_th_check":rho_th_check, 'g':g}

    scipy.io.savemat(f'../matlab/coh_data_NL = {N-c},NM = {c}_g{index+1}.mat',data_dict)

Starting with g = 0.001


C:\Users\DELL\AppData\Local\Temp\ipykernel_42664\1603936380.py:50: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,f,args=(tb,beta,mu2,gamma1),limit=limit_value)[0]


1.3921569049679551e-14
Starting with g = 0.0021544346900318843
3.669377733433922e-14
Starting with g = 0.004641588833612777
1.3805877444715573e-14
Starting with g = 0.01
1.7980534848688275e-14
Starting with g = 0.021544346900318832
3.954851095464103e-15
Starting with g = 0.046415888336127774
2.6321510871429535e-15
Starting with g = 0.1
2.778607236939854e-15
Starting with g = 0.21544346900318823
1.7632356210149562e-15
Starting with g = 0.46415888336127775
5.909270640513128e-16
Starting with g = 1.0
9.617175008075386e-16
